<a href="https://colab.research.google.com/github/sevilcaliskan/hepsiburadaRecommendationTeamCase/blob/main/nameSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving meta.json to meta.json


In [2]:
import pandas as pd
import numpy as np
import json
import io

data = next(iter(uploaded.values()))
d = json.loads(data.decode())

meta = pd.json_normalize(d, record_path =['meta'])
meta

,productid,brand,category,subcategory,name
0,HBV00000AX6LR,Palette,Kişisel Bakım,Saç Bakımı,Palette Kalıcı Doğal Renkler 10-4 PAPATYA
1,HBV00000BSAQG,Best,Pet Shop,Kedi,Best Pet Jöle İçinde Parça Etli Somonlu Konser...
2,HBV00000JUHBA,Tarım Kredi,Temel Gıda,"Bakliyat, Pirinç, Makarna",Türkiye Tarım Kredi Koop.Yeşil Mercimek 1 kg
3,HBV00000NE0QI,Namet,"Et, Balık, Şarküteri",Şarküteri,Namet Fıstıklı Macar Salam 100 gr
4,HBV00000NE0UQ,Muratbey,Kahvaltılık ve Süt,Peynir,Muratbey Burgu Peyniri 250 gr
...,...,...,...,...,...
10231,ZYHPREISBBKL008,Reis,Temel Gıda,"Bakliyat, Pirinç, Makarna",Reis Gönen Baldo Pirinç 1 kg
10232,ZYNES11470137,Nescafe,İçecekler,Kahve,Nescafé Classic Çözünebilir Kahve 200gr Ekopaket
10233,ZYPAREX1909309,Parex,Ev Bakım ve Temizlik,Ev Temizlik Ürünleri,Parex Powerball Çizmez Ovma Teli
10234,ZYPAREX2107986,Parex,Ev Bakım ve Temizlik,Yüzey Temizleyiciler,Parex Tornado Döner Başlıklı Temizlik Seti


In [46]:
meta = meta[~meta.subcategory.isnull()]

In [3]:
!pip install -V sentence-transformers

     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 2.5MB 37.7MB/s 
     |████████████████████████████████| 1.2MB 40.3MB/s 
     |████████████████████████████████| 901kB 50.1MB/s 
     |████████████████████████████████| 3.3MB 37.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=3decda1a06609a703bd667b7e1d98a16f09b467d8bae53b2615fc5a93d14e16b
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v1', device='cuda')

In [49]:
subCategories = meta.subcategory.unique().tolist()
subCategories

['Saç Bakımı',
 'Kedi',
 'Bakliyat, Pirinç, Makarna',
 'Şarküteri',
 'Peynir',
 'Yoğurt',
 'Su',
 'Tatlı ve Tuzlu Kurabiyeler',
 'Meyve',
 'Ekmekler',
 'Gazete ve Dergi',
 'Mutfak Ve Banyo Ürünleri',
 'Çikolata, Gofret ve Barlar',
 'Tatlı Malzemeleri',
 'Oda Kokusu ve Koku Gidericiler',
 'Devam Sütleri ve Ek Gıdalar',
 'Oyuncak',
 'Yapıştırıcı ve Etiketler',
 'Haşere Öldürücüler',
 'Duş Jelleri ve Sabunlar',
 'Ağız Bakım',
 'Pil',
 'Hijyenik Pedler',
 'Konserve',
 'Hazır Yemekler',
 'Kalemler',
 'Tıraş Ürünleri',
 'El, Yüz ve Vücut Bakımı',
 'Diyet Ürünler',
 'Dondurulmuş Sebze, Meyve',
 'Çay',
 'Gazsız İçecekler',
 'Bisküvi ve Kekler',
 'Sıvı Yağ',
 'Süt',
 'Bebek Bakım ve Sağlığı',
 'Ev Temizlik Ürünleri',
 'Meyve Suyu',
 'Kahve',
 'Balık ve Deniz Mahsülleri',
 'Sebze',
 'Kümes Hayvanları',
 'Islak Mendil',
 'Krem Çikolata ve Ezme',
 'Bebek Bezi',
 'Baharat, Harç ve Bulyon',
 'Parfüm, Deodorant',
 'Yufka, Taze Hamur ve Mayalar',
 'Kağıt Ürünler',
 'Makyaj Ürünleri',
 'Kap Dondurma',


In [50]:
lenCats = []
for i in subCategories:
  lenCats.append(meta[meta.subcategory == i].shape[0])

In [56]:
subCategories

['Saç Bakımı',
 'Kedi',
 'Bakliyat, Pirinç, Makarna',
 'Şarküteri',
 'Peynir',
 'Yoğurt',
 'Su',
 'Tatlı ve Tuzlu Kurabiyeler',
 'Meyve',
 'Ekmekler',
 'Gazete ve Dergi',
 'Mutfak Ve Banyo Ürünleri',
 'Çikolata, Gofret ve Barlar',
 'Tatlı Malzemeleri',
 'Oda Kokusu ve Koku Gidericiler',
 'Devam Sütleri ve Ek Gıdalar',
 'Oyuncak',
 'Yapıştırıcı ve Etiketler',
 'Haşere Öldürücüler',
 'Duş Jelleri ve Sabunlar',
 'Ağız Bakım',
 'Pil',
 'Hijyenik Pedler',
 'Konserve',
 'Hazır Yemekler',
 'Kalemler',
 'Tıraş Ürünleri',
 'El, Yüz ve Vücut Bakımı',
 'Diyet Ürünler',
 'Dondurulmuş Sebze, Meyve',
 'Çay',
 'Gazsız İçecekler',
 'Bisküvi ve Kekler',
 'Sıvı Yağ',
 'Süt',
 'Bebek Bakım ve Sağlığı',
 'Ev Temizlik Ürünleri',
 'Meyve Suyu',
 'Kahve',
 'Balık ve Deniz Mahsülleri',
 'Sebze',
 'Kümes Hayvanları',
 'Islak Mendil',
 'Krem Çikolata ve Ezme',
 'Bebek Bezi',
 'Baharat, Harç ve Bulyon',
 'Parfüm, Deodorant',
 'Yufka, Taze Hamur ve Mayalar',
 'Kağıt Ürünler',
 'Makyaj Ürünleri',
 'Kap Dondurma',


In [57]:
similarity = []
for i in subCategories:
  subcat = meta[meta.subcategory == i ]
  print("Subcategory : "+ i)
  product_pair_map = {}
  for j in range(len(subcat.name)):
      product_A = subcat.iloc[j]['name']
      embed_A = model.encode(product_A)
      for m in range(len(subcat.name)):
          product_B = subcat.iloc[m]['name']
          #print("product_A = ", product_A , " Product_B =", product_B)
          if product_A != product_B:
              rows = []
              rows.append(subcat.iloc[j]['productid']) 
              rows.append(product_A) 
              pair_key = str(product_A) + '#' + str(product_B)
              if product_B > product_A:
                  pair_key = str(product_B) + "#" + str(product_A)
              if product_pair_map.get(pair_key) == None:
                  embed_B = model.encode(product_B)
                  cos_sim_A_B = util.cos_sim(embed_A, embed_B)
                  product_pair_map[pair_key] = cos_sim_A_B.item()

                  rows.append(subcat.iloc[m]['productid']) 
                  rows.append(product_B)
                  rows.append(cos_sim_A_B.item())
                  #print(rows)
                  similarity.append(rows)
  print("Done")
sim_df2 = pd.DataFrame(similarity, columns = ['Product_A_id','Product_A',  'Product_B', 'Product_B_id','Similarity'])
sim_df2.head()

Subcategory : Saç Bakımı
Done
Subcategory : Kedi
Done
Subcategory : Bakliyat, Pirinç, Makarna
Done
Subcategory : Şarküteri
Done
Subcategory : Peynir
Done
Subcategory : Yoğurt
Done
Subcategory : Su
Done
Subcategory : Tatlı ve Tuzlu Kurabiyeler
Done
Subcategory : Meyve
Done
Subcategory : Ekmekler
Done
Subcategory : Gazete ve Dergi
Done
Subcategory : Mutfak Ve Banyo Ürünleri
Done
Subcategory : Çikolata, Gofret ve Barlar
Done
Subcategory : Tatlı Malzemeleri
Done
Subcategory : Oda Kokusu ve Koku Gidericiler
Done
Subcategory : Devam Sütleri ve Ek Gıdalar
Done
Subcategory : Oyuncak
Done
Subcategory : Yapıştırıcı ve Etiketler
Done
Subcategory : Haşere Öldürücüler
Done
Subcategory : Duş Jelleri ve Sabunlar
Done
Subcategory : Ağız Bakım
Done
Subcategory : Pil
Done
Subcategory : Hijyenik Pedler
Done
Subcategory : Konserve
Done
Subcategory : Hazır Yemekler
Done
Subcategory : Kalemler
Done
Subcategory : Tıraş Ürünleri
Done
Subcategory : El, Yüz ve Vücut Bakımı
Done
Subcategory : Diyet Ürünler
Done


,Product_A_id,Product_A,Product_B,Product_B_id,Similarity
0,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000PV87H,Gliss Ultimate Oil Elixir Saç Kremi 400 ml,0.206611
1,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000SP88B,Qenzym Q10 Şampuan 360 ml,0.222637
2,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000U2AZA,Gliss 4'ü 1 Arada Saç Bakım Kürü Besleyici Yıp...,0.193493
3,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,SGBA1911,L'oréal Paris Excellence Creme Saç Boyası 6.30...,0.201084
4,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,SGPAN972028,Pantene Pro-V Nem Terapisi Şampuanı 500 ml,0.148332


In [59]:
sim_df2.sort_values(by = 'Similarity', ascending= False)

,Product_A_id,Product_A,Product_B,Product_B_id,Similarity
623744,HBV00000PQOYO,Parex Rengarenk Temizlik Bezi 5'li,ZYPAREX2107172,Parex Rengarenk Temizlik Bezi 5 'li,1.000000
719773,ZYECZACI8664019,Silen Ultra Tuvalet Kağıdı 32'li,HBV00000PVC8A,Silen Ultra Tuvalet Kağıdı 32 'li,1.000000
436695,HBV00000PVQDP,Ipana 3 Boyutlu Beyazlık Therapy Diş Macunu Di...,HBV00000INLAK,Ipana 3 Boyutlu Beyazlık Therapy Diş Minesi Ba...,0.999334
616101,HBV00000IKMZH,Johnson's Baby Losyon Bedtime 300 ml,HBV00000PV8R8,Johnson's Baby Bedtime Losyon 300 ml,0.998654
721710,HBV00000PVA85,Nivea 3'ü 1 Arada Makyaj Temizleme Mendili Kur...,SGEZ9102365,Nivea 3'Ü 1 Arada Makyaj Temizleme Mendili Kur...,0.998276
...,...,...,...,...,...
438280,HBV00000PQOY9,Cross Action 2'li Yedek Başlık Siyah,SGSER211201,Sensodyne Fluoride Florürlü Diş Macunu 100 ml.,-0.082259
795421,HBV0000048150,Scotch Brite Kullan-At Islak Mutfak Bezi 50 Ya...,HBV00000PVD2H,Vileda Turbo Yer Temizlik Sistemi,-0.084446
795457,HBV0000048150,Scotch Brite Kullan-At Islak Mutfak Bezi 50 Ya...,HBV00000PVD01,Vileda Çekçek Sistem,-0.099519
886415,HBV00000LZI7D,Gurur Ve ÖnyargıŞebnem İşigüzel'İn Önsözüyle -...,HBV00000DFYMB,365 Aktivite,-0.103149


In [64]:
sim_df2[sim_df2.Product_A_id == 'HBV00000AX6LR'].sort_values(by = 'Similarity', ascending = False)

,Product_A_id,Product_A,Product_B,Product_B_id,Similarity
398,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000AX6L5,Palette Kalıcı Doğal Renkler 10-0 AÇIK SARI,0.878606
102,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000AX6LH,Palette Kalıcı Doğal Renkler 4-0 KAHVE,0.862035
375,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000AX6LT,Palette Kalıcı Doğal Renkler 9-4 SAHRA SARISI,0.837477
34,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000AX6KZ,Palette Kalıcı Doğal Renkler 4-60 AÇIK ÇİKOLAT...,0.831735
338,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000AX6LV,Palette Kalıcı Doğal Renkler 1-0 SİYAH,0.780855
...,...,...,...,...,...
304,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV000006THC6,Pantene 3 Minute Miracle 200 ml Saç Dökülmeler...,0.046042
212,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000EIRGB,Pantene 3 Minute Miracle Saç Bakım Kremi Super...,0.039478
104,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000PVB4T,Saç Bakım Kremi 3 Minute Mıracle Saç Dökülmele...,0.039159
81,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,HBV00000SP7ZO,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,0.023743


In [79]:
sim_df2[sim_df2.Product_A_id == 'HBV00000SP7ZS'].sort_values(by = 'Similarity', ascending = False)

,Product_A_id,Product_A,Product_B,Product_B_id,Similarity
68124,HBV00000SP7ZS,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,HBV00000SP7ZQ,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,0.954327
68139,HBV00000SP7ZS,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,HBV00000U27I0,Head & Shoulders Şampuan 250 ml Mentol Ferahlığı,0.732078
68057,HBV00000SP7ZS,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,HBV00000QU40J,Head & Shoulders Şampuan Besleyici Bakım 450 ml,0.726996
68083,HBV00000SP7ZS,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,HBV00000QU403,Head & Shoulders Men Ultra Anında Saç Derisi R...,0.714442
68019,HBV00000SP7ZS,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,HBV00000U27HY,Head & Shoulders Şampuan 250 ml Erkekler Özel ...,0.710967
...,...,...,...,...,...
68259,HBV00000SP7ZS,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,HBV000006BTG5,Colour Specialist Küllü Açık Kumral 8.1,-0.009474
68077,HBV00000SP7ZS,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,HBV000003WZOW,Palette Göz Alıcı Renkler 5-68 KESTANE,-0.015514
68131,HBV00000SP7ZS,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,HBV0000037Q68,Palette Göz Alıcı Renkler 8-0 KOYU SARI,-0.029339
68251,HBV00000SP7ZS,Head & Shoulders Kepeğe Karşı Etkili Şampuan S...,HBV00000PVBXJ,"Krem Karamel 7,57",-0.030343


In [80]:
from google.colab import files
sim_df2[['Product_A_id', 'Product_B','Similarity']].to_csv('all_categories_backup.txt', sep = ";") 
files.download('all_categories_backup.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>